# Simulation
## 1 Setting up a simulator using configuration saved on file
## 2 Running a simulation
## 3 Changing simulation settings
### 3.1 Simulation period (Time-axis)
### 3.2 Model parameters
#### 3.2.1 Region parameters
#### 3.2.2 Catchment parameters
### 3.3 Initial state
#### 3.3.1 Setting spatially invariable state
#### 3.3.2 Setting state using a warm-up period
## 4 Running a simulation with updated settings
## 5 Activating simulation only for selected catchments
## 6 Setting a different input dataset (source)
## 7 Activating state collection
## 8 Extracting results

In [ ]:
# Simulation
import os

from shyft import api
from shyft.orchestration.configuration.yaml_configs import YAMLSimConfig
from shyft.orchestration.simulators.config_simulator import ConfigSimulator

In [ ]:
# 1.Setting up a simulator using configuration saved on file
config_dir = "D:/users/ysa/shyft_config/yaml"
config_file = os.path.join(config_dir, "simulation_config_calibration-run.yaml")
region_name = "Nea-Nidelv"
cfg = YAMLSimConfig(config_file, region_name)
simulator = ConfigSimulator(cfg)

In [ ]:
# 2.Running a simulation
region_model_id = simulator.region_model_id
interpolation_id = simulator.interpolation_id

region_model_repo = simulator.region_model_repository
interpolation_param_repo = simulator.ip_repos
geo_ts_repo = simulator.geo_ts_repository
initial_state_repo = simulator.initial_state_repo

geo_ts_names = ("temperature", "wind_speed", "precipitation", "relative_humidity", "radiation")

#region_model = region_model_repo.get_region_model(region_model_id)
region_model = simulator.region_model

epsg = region_model.bounding_region.epsg()
bbox = region_model.bounding_region.bounding_box(epsg)
period = region_model.time_axis.total_period()

sources = geo_ts_repo.get_timeseries(geo_ts_names, period, geo_location_criteria=bbox)


def get_region_env(sources_):
    region_env_ = api.ARegionEnvironment()
    region_env_.temperature = sources_["temperature"]
    region_env_.precipitation = sources_["precipitation"]
    region_env_.radiation = sources_["radiation"]
    region_env_.wind_speed = sources_["wind_speed"]
    region_env_.rel_hum = sources_["relative_humidity"]
    return region_env_

interpolation_parameters = interpolation_param_repo.get_parameters(interpolation_id)
region_env = get_region_env(sources)
#region_model.run_interpolation(interpolation_parameters, region_model.time_axis, region_env)
region_model.interpolate(interpolation_parameters, region_env)


def get_init_state_from_repo(initial_state_repo_, region_model_id_=None, timestamp=None):
    state_id = 0
    if hasattr(initial_state_repo_, 'n'):  # No stored state, generated on-the-fly
        initial_state_repo_.n = region_model.size()
    else:
        states = initial_state_repo_.find_state(
            region_model_id_criteria=region_model_id_,
            utc_timestamp_criteria=timestamp)
        if len(states) > 0:
            state_id = states[0].state_id  # most_recent_state i.e. <= start time
        else:
            raise Exception('No initial state matching criteria.')
    return initial_state_repo_.get_state(state_id)

init_state = get_init_state_from_repo(initial_state_repo, region_model_id_=region_model_id, timestamp=region_model.time_axis.start)
#region_model.initial_state = init_state
region_model.set_states(init_state)
region_model.run_cells()

# Running a simulation using orchestration function
#simulator.run()

In [ ]:
# 2.Changing simulation settings
# 2.1Simulation period (Time-axis)
ta_orig = region_model.time_axis
ta_new = api.Timeaxis(ta_orig.start-ta_orig.delta_t*ta_orig.n, ta_orig.delta_t, ta_orig.n)
region_model.initialize_cell_environment(ta_new)

init_state = get_init_state_from_repo(initial_state_repo, region_model_id_=region_model_id, timestamp=region_model.time_axis.start)
region_model.initial_state = init_state

In [ ]:
# 2.2Model parameters
# 2.2.1Region parameters
param_reg = region_model.get_region_parameter()
param_reg.gs.tx *= 1.5
param_reg_default = region_model.parameter_t()
region_model.set_region_parameter(param_reg_default)

In [ ]:
# 2.2.2Catchment parameters
catch_id = region_model.catchment_id_map[0]
region_model.has_catchment_parameter(catch_id)
param_reg_default.gs.tx *= 2
region_model.set_catchment_parameter(catch_id, param_reg_default)
param_catch = region_model.parameter_t(param_reg_default)
param_catch.gs.tx *= 2
region_model.set_catchment_parameter(catch_id, param_catch)
region_model.has_catchment_parameter(catch_id)

In [ ]:
# 2.3Initial state
# 2.3.1Setting spatially invariable state
from shyft.repository.generated_state_repository import GeneratedStateRepository
init_values = {'gamma_snow':
                   {'acc_melt': 0.0,
                    'albedo': 0.65,
                    'alpha': 6.25,
                    'iso_pot_energy': 0.0,
                    'lwc': 0.1,
                    'sdc_melt_mean': 0.0,
                    'surface_heat': 30000.0,
                    'temp_swe': 0.0},
              'kirchner':
                  {'q': 0.01}
              }
model_type = region_model.__class__
uniform_initial_state_repo = GeneratedStateRepository(model_type, init_values)
init_state_uniform = get_init_state_from_repo(uniform_initial_state_repo)
region_model.initial_state = init_state_uniform

In [ ]:
# 2.3.2Setting state using a warm-up period
ta_main_sim = region_model.time_axis
cal = api.Calendar()
warm_up_period = cal.YEAR
ta_warm_up = api.Timeaxis(ta_main_sim.start-cal.YEAR, ta_main_sim.delta_t, int(cal.YEAR/ta_main_sim.delta_t))
region_model_warm_up = region_model.__class__(region_model.extract_geo_cell_data(), region_model.get_region_parameter())
region_model_warm_up.initialize_cell_environment(ta_warm_up)
period = region_model_warm_up.time_axis.total_period()
sources = geo_ts_repo.get_timeseries(geo_ts_names, period, geo_location_criteria=bbox)
region_env = get_region_env(sources)
region_model_warm_up.interpolate(interpolation_parameters, region_env)
init_state_for_warm_up = get_init_state_from_repo(uniform_initial_state_repo)
region_model_warm_up.set_states(init_state_for_warm_up)
region_model_warm_up.run_cells()
init_state_from_warm_up = region_model_warm_up.__class__.state_t.vector_t()
region_model_warm_up.get_states(init_state_from_warm_up)
region_model.initial_state = init_state_from_warm_up

In [ ]:
# 3.Running a simulation with updated settings
region_model.revert_to_initial_state()
region_model.run_cells()

In [ ]:
# 4.Activating simulation only for selected catchments
catch_ids_active = region_model.catchment_id_map[0:2]
region_model.set_catchment_calculation_filter(catch_ids_active)
[region_model.is_calculated(cid) for cid in region_model.catchment_id_map]
region_model.set_catchment_calculation_filter(region_model.catchment_id_map)

In [ ]:
# 5.Setting a different input dataset (source)
from shyft.orchestration.configuration.yaml_configs import YamlContent
#new_datasets_config_file = <path_to_file>
new_datasets_config_file = cfg.datasets_config_file # using same file just as example
datasets_config = YamlContent(new_datasets_config_file)
geo_ts_repo = YAMLSimConfig.construct_geots_repo(datasets_config, epsg=epsg)

In [ ]:
# 6.Activating state collection
catch_id_with_state = region_model.catchment_id_map[0]
region_model.set_state_collection(catch_id_with_state, True)
region_model.set_state_collection(-1, True)

In [ ]:
# 7.Extracting results
# Refer to separate notebook on data_extraction